In [4]:
from data_loader import data_reader, colnames, colnames_bid
import pandas as pd

In [24]:
imp = data_reader('data\\imp.20131020.txt', colnames, 
                verbose=False, fuck_parsing=True)
bid = data_reader('data\\bid.20131020.txt', colnames_bid, verbose=False, fuck_parsing=True)
              

In [25]:

#%%
imp[['bid_id', 'timestamp', 'bidding_price', 'paying_price']].bidding_price.unique()

array([277, 294], dtype=int64)

In [27]:
bid.columns

Index(['bid_id', 'timestamp', 'ipinyou_id', 'user_agent', 'ip_address',
       'region_id', 'city_id', 'ad_exchange', 'domain', 'url',
       'anonymous_url_id', 'ad_slot_id', 'ad_slot_width', 'ad_slot_height',
       'ad_slot_visibility', 'ad_slot', 'ad_slot_floor_price', 'creative_id',
       'bidding_price', 'advertiser_id', 'user_tags'],
      dtype='object')